## IMPORTING THE REQUIRED LIBRARIES

In [1]:
import os
import numpy as np
from PIL import Image
import pandas as pd
from tensorflow.keras.models import load_model

## PUTTING THE PIXELS OF EVERY PHOTO IN A DATAFRAME

In [2]:
def importImages(path):

    images = os.listdir(path)

    data = list()

    for image in images:

        row = list()

        start = image.find('') + len('')
        end = image.find('.jpg')
        name = image[start:end]
        row.append(str(name))

        imageObject = Image.open(f'{path}/{image}')
        imageResized = imageObject.resize((48, 48))
        imageArray = np.asarray(imageResized)

        pixelsRows = list()

        for pixelsRow in imageArray:
            for pixel in pixelsRow:
                pixelsRows.append(pixel[0])
    
        row.append(pixelsRows)

        data.append(row)
        
    return pd.DataFrame(data = data, columns = ['name', 'pixels'])

images = importImages('facesImages')

## PREPARING THE PIXELS COLUMN FOR THE AGE NEURAL NETWORK

In [3]:
def formatPixelsAge(pixels: str) -> np.array:
    return np.asarray(pixels, dtype = np.uint8).reshape(48, 48)

pixelsAge = images['pixels'].apply(formatPixelsAge)
pixelsAge = np.reshape(pixelsAge.to_list(), (len(pixelsAge), 48, 48, 1)) / 255.0

## IMPORTING THE CLASSIFIER AND PREDICTING THE AGE OF THE FACES

In [4]:
ageClassifier = load_model('../models/ageFull.h5')
agePredictions = ageClassifier.predict(pixelsAge)

## PREPARING THE PIXELS COLUMN FOR THE GENDER NEURAL NETWORK

In [5]:
def formatPixelsGender(pixels: str) -> np.array:

    pixelsArrayReshaped = np.asarray(pixels, dtype = np.uint8).reshape(48, 48)
    pixelsNormalized = pixelsArrayReshaped / 255.0

    return pixelsNormalized

pixelsGender = images['pixels'].apply(formatPixelsGender)
pixelsGender = np.reshape(pixelsGender.to_list(), (len(pixelsGender), 48, 48, 1)) / 255.0

## IMPORTING THE CLASSIFIER AND PREDICTING THE GENDER OF THE FACES

In [6]:
genderClassifier = load_model('../models/genderFull.h5')
genderPredictions = genderClassifier.predict(pixelsGender)

## INSERTING ALL THE PREDICTIONS IN THE IMAGES DATAFRAME

In [7]:
images['gender'] = [genderPrediction[0] for genderPrediction in genderPredictions]
images['age'] = [agePrediction[0] for agePrediction in agePredictions]

## PRESENTING THE PREDICTIONS

In [8]:
images

,name,pixels,gender,age
0,erin,"[148, 149, 152, 152, 153, 154, 155, 156, 160, ...",0.314776,5.916344
1,frederico,"[182, 182, 180, 179, 175, 171, 163, 154, 142, ...",0.790406,0.000000
2,gonzalo,"[224, 224, 226, 227, 225, 225, 227, 227, 226, ...",0.996409,39.385677
3,hamza,"[183, 183, 180, 178, 173, 171, 170, 173, 175, ...",0.906473,19.491829
4,ines,"[204, 203, 202, 200, 199, 194, 181, 134, 162, ...",0.290397,43.105206
5,ishmael,"[218, 218, 218, 218, 217, 217, 217, 216, 216, ...",0.037600,14.236519
6,jan,"[217, 227, 222, 229, 216, 173, 124, 169, 230, ...",0.994866,33.905205
7,jorien,"[151, 97, 96, 102, 104, 104, 106, 109, 108, 13...",0.076819,12.977506


In [9]:
gender = [0, 1, 1, 1, 0, 1, 1, 0]
age = [50, 28, 23, 28, 22, 32, 59, 24]

from sklearn.metrics import accuracy_score, r2_score

print(f'GENDER ACCURACY: {accuracy_score(gender, round(images["gender"], 0))}')
print(f'AGE ACCURACY: {r2_score(age, images["age"])}')

GENDER ACCURACY: 0.875
AGE ACCURACY: -2.4766324098963755
